## Decision Tree
*** Date : 2019-3-16 ***

*** Author : wwt117@163.com ***

*** Reference : *** <br/> 
*** 1.《 Machine Learning in Action 》(chapter 3)  -- Peter Harrington *** <br/> 
*** 2. http://blog.csdn.net/c406495762  -- Jack-Cui*** <br/> 

#### DT原理

根据最好特征对数据集进行划分，对划分后的数据子集同样进行上述操作，直到数据子集中的数据都属于同一类型。

#### DT特点
- 优点：计算复杂度不高，输出结果易于理解，对中间值缺失不敏感，可以处理不相关特征数据
- 缺点：容易过拟合

#### 数据集划分
- 划分依据：信息增益、信息增益比、基尼指数
- 划分方法：ID3（以下使用的方法）、C4.5、CART



#### 样例1：贷款申请的批准
* ***数据来源：《统计学习方法》 --李航***

<center>**贷款申请样本数据表**<center>

|ID|年龄|有工作|有自己的房子|信贷情况|类别|
|:------:|:------:|:------:|:------:|:------:|:------:|
|1|青年|否|否|一般|否|
|2|青年|否|否|好|否|
|3|青年|是|否|好|是|
|4|青年|是|是|一般|是|
|5|青年|否|否|一般|否|
|6|中年|否|否|一般|否|
|7|中年|否|否|好|否|
|8|中年|是|是|好|是|
|9|中年|否|是|非常好|是|
|10|中年|否|是|非常好|是|
|11|老年|否|是|非常好|是|
|12|老年|否|是|好|是|
|13|老年|是|否|好|是|
|14|老年|是|否|非常好|是|
|15|老年|否|否|一般|否|


In [16]:
#----------------------------------
#  年龄 ：青年--0 中年--1 老年--2
#  有工作 ：否--0 是--1
#  有自己的房子 ：否--0 是--1
#  信贷情况 ：一般--0 好--1 非常好--2
#  类别 ：否--no 是--yes
#-----------------------------------

from math import log 
import operator 


def create_dataset():
    """创建数据集
    
    输出参数：
    - - - - - - 
    data:
        样本集
    label:
        特征集
    
    """
    data = [[0, 0, 0, 0, 'no'],   
            [0, 0, 0, 1, 'no'],
            [0, 1, 0, 1, 'yes'],
            [0, 1, 1, 0, 'yes'],
            [0, 0, 0, 0, 'no'],
            [1, 0, 0, 0, 'no'],
            [1, 0, 0, 1, 'no'],
            [1, 1, 1, 1, 'yes'],
            [1, 0, 1, 2, 'yes'],
            [1, 0, 1, 2, 'yes'],
            [2, 0, 1, 2, 'yes'],
            [2, 0, 1, 1, 'yes'],
            [2, 1, 0, 1, 'yes'],
            [2, 1, 0, 2, 'yes'],
            [2, 0, 0, 0, 'no']]
    label = ['年龄', '有工作', '有自己的房子', '信贷情况']
    return data, label            


def cal_ent(data):
    """计算香农熵
    
    输入参数：
    - - - - - 
    data:
        样本集
    
    输出参数：
    - - - - - 
    ent:
        样本集的熵
    """
    # 样本个数
    n = len(data)
    
    # 用于存储每种标签次数
    label_count = {}

    # 统计标签次数
    for sample in data:
        curlabel = sample[-1]
        if curlabel not in label_count.keys():
            label_count[curlabel] = 0
        label_count[curlabel] += 1
    
    ent = 0
    # 计算熵
    for key in label_count:
        prob = float(label_count[key]) / n;
        ent -= prob * log(prob,2)
    return ent
        
        

def splitdataset(data,feature,value):
    """划分数据集
    
    输入参数：
    - - - - - 
    data:
        样本集
    feature:
        特征
    value:
        特征的取值

    输出参数：
    - - - - - 
    rdata:
        去掉某个特征后的数据集
    """
    rdata = []
    for sample in data:
        if sample[feature] == value:
            # 样本中去掉当前划分特征
            rsample = sample[:feature]
            rsample.extend(sample[feature+1:])
            rdata.append(rsample)
    return rdata
        
        
def choosefeature(data):
    """选择最优特征
    
    输入参数：
    - - - - - 
    data：
        样本集
        
    输出参数：
    - - - - - 
    bestfeature:
        最优特征
    
    """
    # 特征数量
    num_feature = len(data[0]) - 1    
    
    # 整个数据集的熵
    ent0 = cal_ent(data)
    
    # 信息增益
    best_gain = 0.0
    
    # 最优特征
    bestfeature = -1
    
    # 遍历所有样本找到对应信息增益最大的特征
    for i in range(num_feature):
        # 所有样本的第i个特征列表
        feature_list = [sample[i] for sample in data] 
        value_list = set(feature_list)
        # 每个子集的条件熵
        ent1 = 0
        # 计算信息增益
        for value in value_list:
            subdata = splitdataset(data,i,value)
            prob = len(subdata) / float(len(data))
            ent1 += prob * cal_ent(subdata)
        gain = ent0 - ent1
        print("第%d个特征的增益为%.3f" % (i,gain))
        if(gain > best_gain):
            best_gain = gain
            bestfeature = i
    print("最优特征为%d" % i)
    return bestfeature
    
def majoritycnt(classlist):
    """统计classlist中出现最多的元素
    
    输入参数：
    - - - - - 
    classlist:
        类标签列表
        
    输出参数：
    - - - - -  
    sorted_list[0][0]:
        出现次数最多的元素
    
    """
    classcount = {}
    # 统计classlist中每个元素出现次数
    for vote in classlist:
        if vote not in classcount.key():
            classcount[vote] = 0
        classcount[vote] += 1
        
    # 字典降序排序
    sorted_list = sorted(classcount.items(), key = operator.itemgetter(1), reverse = True)
    
    # 返回最多的类
    return sorted_list[0][0]

def create_tree(data,label):
    """创建决策树
    
    输入参数：
    - - - - - 
    data:
        样本集
    label:
        标签
    
    输出参数：
    - - - - - 
    tree0:
        建立的决策树
    """
    # 取出分类标签（是否放贷）
    class_list = [sample[-1] for sample in data]
    
    # 如果类别只有一种，则停止划分
    if class_list.count(class_list[0]) == len(class_list):
        return class_list[0]
    
    # 遍历完所有特征时返回出现次数最多的类别
    if len(data[0]) == 1:
        return majoritycnt(class_list)
    
    # 选择最优特征
    bestfeature = choosefeature(data)
    
    # 最优特征的标签
    bestlabel = label[bestfeature]

    # 根据最优特征生成树
    tree0 = {bestlabel:{}}
    
    # 删除使用的标签
    del(label[bestfeature])
    
    feature_list = [sample[bestfeature] for sample in data]
    value_list = set(feature_list)
    for value in value_list:
        sublabel = label[:]
        tree0[bestlabel][value] = create_tree(splitdataset(data,bestfeature,value),sublabel)
    return tree0

     
        
if __name__=="__main__":
    # 创建数据
    data,label = create_dataset()
    
    # 最优特征标签列表
    feat_label = ['有自己的房子', '有工作']
    
    # 创建决策树
    tree0 = create_tree(data,label)
    print(tree0)
    

第0个特征的增益为0.083
第1个特征的增益为0.324
第2个特征的增益为0.420
第3个特征的增益为0.363
最优特征为3
第0个特征的增益为0.252
第1个特征的增益为0.918
第2个特征的增益为0.474
最优特征为2
{'有自己的房子': {0: {'有工作': {0: 'no', 1: 'yes'}}, 1: 'yes'}}
放贷


#### 2. 使用决策树进行分类

In [17]:
def classify(tree,feat_label,test_data):
    """使用决策树进行分类
    
    输入参数：
    - - - - - - 
    tree:
        已生成的决策树
    featlabel:
        存储最优特征标签
    testdata:
        待分类数据
        
    输出参数：
    - - - - - -
    class_label:
        分类结果
    """
    # 获取决策树节点
    first_str = next(iter(tree))
    
    # 下一个字典
    second_dict = tree[first_str]
    
    # 找到特征索引值
    feat_index = feat_label.index(first_str)
    
    # 遍历树
    for key in second_dict.keys():
        if test_data[feat_index] == key:
            if type(second_dict[key]).__name__ == 'dict':
                class_label = classify(second_dict[key],feat_label,test_data)
            else:
                class_label = second_dict[key]
    return class_label



if __name__=="__main__":
    
    # 待分类数据
    test_data = [0,1]
    
    # 分类结果
    result = classify(tree0,feat_label,test_data)
    
    # 判断是否放贷
    if result == 'yes':
        print('放贷')
    else:
        print('不放贷')
        

放贷


#### 3. 决策树序列化

In [26]:
import pickle

def store_tree(tree,filename):
    """
    存储决策树
    
    输入参数：
    - - - - - 
    tree:
        待存储的决策树
    filename:
        存储树的文件路径
    """
    with open(filename,'wb') as fw:
        pickle.dump(tree,fw)

        
        
def read_tree(filename):
    """
    读取决策树
    
    输入参数：
    - - - - -
    filename:
        存储树的文件路径
        
    输出参数：
    - - - - -     
    tree:
        需要读取的决策树
    """
    fr = open(filename,'rb')
    return pickle.load(fr)
        
        
if __name__ == '__main__':
    # 存储树的文件路径
    filename = 'DT_storage.txt'
    
    # 存储决策树
    store_tree(tree0,filename)
    print("决策树文件存储在:\n%s" % filename)
    
    # 读取决策树
    tree1 =read_tree(filename)
    
    # 输出决策树
    print("读取的决策树为:\n")
    print(tree1)
    

决策树文件存储在:
DT_storage.txt
读取的决策树为:

{'有自己的房子': {0: {'有工作': {0: 'no', 1: 'yes'}}, 1: 'yes'}}
